## 🛠️ **ETL (Extract, Transform, Load)**



En este ipynb vamos a Extraer los archivos y cargarlos en parquet. Luego en otro archivo haremos las transformaciones.

Tambien vamos a estar utilizando la funcion personalizada personalizada `data_type_check` invocada desde `data_utils.py` para dejar un vistazo rápido del dataframe y  poder observar:
- Variables categóricas
- Variables numéricas
- Dimensiones del dataframe
- Nulos
- Tipos de datos
- Informacion acerca de los datos faltantes o nulos de cada columna    


####  **Importamos las librerías que vamos a usar**


In [2]:
import gc
import os
import warnings

import pandas as pd
from data_utils import data_type_check

warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns

___

## Dataset Yelp

### business.pkl

In [6]:
business = pd.read_pickle('../0_Dataset/Data_Sucia/business.pkl') #'../0_Dataset/Data_Sucia/Yelp/business.pkl'
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

Tenemos columnas duplicadas, procedemos a quitarlas.


In [7]:
business = business.loc[:, ~business.columns.duplicated()]
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


Verificamos los nulos y duplicados de categories

In [8]:
# Cuenta los duplicados y nulos en la columna 'categories' antes de cualquier eliminación
duplicados_inicial = business['categories'].duplicated().sum()
nulos_inicial = business['categories'].isna().sum()

print("Antes: Cantidad de duplicados en 'categories':", duplicados_inicial)
print("Antes: Cantidad de NaNs en 'categories':", nulos_inicial)

# Elimina las filas con valores NaN en la columna 'categories'
business = business.dropna(subset=['categories'])

# Cuenta los duplicados y nulos en la columna 'categories' después de eliminar NaNs
duplicados_despues_nulos = business['categories'].duplicated().sum()
nulos_despues_nulos = business['categories'].isna().sum()

print("Después: Cantidad de NaNs en 'categories' ", nulos_despues_nulos)


'''
________________________________________________________________
FUTURO: Descomentar o borrar
Elimina las filas duplicadas basadas en la columna 'categories'
business = business.drop_duplicates(subset=['categories'])  
_________________________________________________________________  
'''

# Cuenta los duplicados en la columna 'categories' después de eliminar duplicados
duplicados_final = business['categories'].duplicated().sum()

print("Después: Cantidad de duplicados en 'categories'", duplicados_final)


Antes: Cantidad de duplicados en 'categories': 67185
Antes: Cantidad de NaNs en 'categories': 103
Después: Cantidad de NaNs en 'categories'  0
Después: Cantidad de duplicados en 'categories' 67083


#### **📤 LOAD**

In [10]:
#guardar en parquet
business.to_parquet("../0_Dataset/Data_Sucia/Yelp/business.parquet", engine="pyarrow")

### review.json

In [8]:
# Cantidad de datos por porción
chunk_s = 50000

# Ruta del archivo
archivo = '../0_Dataset/Yelp/review.json'

# Inicializar una lista para almacenar las muestras
muestras = []

# Se carga el JSON en un DataFrame por partes y se toma una muestra de cada parte
for df in pd.read_json(archivo, lines=True, chunksize=chunk_s):
    muestra = df.sample(frac=0.03, random_state=1)  # Ajusta el tamaño de la muestra según tus necesidades
    muestras.append(muestra)

# Concatenar todas las muestras en un único DataFrame
df_muestra_completa = pd.concat(muestras, axis=0, ignore_index=True)


In [9]:
data_type_check(df_muestra_completa)
df_muestra_completa.sample(2)


 Resumen del dataframe:

Dimensiones:  (209708, 9)
       columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0    review_id       100.0      0.0            0          object
1      user_id       100.0      0.0            0          object
2  business_id       100.0      0.0            0          object
3        stars       100.0      0.0            0           int64
4       useful       100.0      0.0            0           int64
5        funny       100.0      0.0            0           int64
6         cool       100.0      0.0            0           int64
7         text       100.0      0.0            0          object
8         date       100.0      0.0            0  datetime64[ns]


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
82716,g_HGbCH5hLtP0wV14D0aVw,dv4qZSuKlUL4YuVQ6i4vCQ,mnN5N-DoS3uAcG5hVHKRBQ,3,3,1,0,I generally only come to Michael's with my fia...,2021-05-13 15:11:16
88987,7A_w9_VjxqLQT-DTFBPddQ,MKt2J_lRy8F8mHwBIyY_KQ,2KHS8Vuao5QZhs2YUE3b5g,1,0,0,0,One star is too generous. I went in to get my ...,2016-06-04 19:08:11



📤 LOAD

In [10]:
# Guardar la muestra en un archivo Parquet
muestra_archivo = '../0_Dataset/Data_Sucia/Yelp/review_reducido.parquet'
df_muestra_completa.to_parquet(muestra_archivo)

### checkin.json

In [11]:
# Ruta del archivo JSON
checkin_path = '../0_Dataset/Yelp/checkin.json'
output_path = '../0_Dataset/Yelp/Data_Sucia/checkin_reducido.parquet'

# Leer JSON
df = pd.read_json(checkin_path, lines=True)


In [12]:
# Estimar la fracción de datos para que el archivo resultante pese menos de 100 MB
# Aquí asumimos que podemos trabajar con el 10% de los datos originales.
# Ajusta la fracción según sea necesario después de verificar el tamaño del archivo resultante.
fraccion = 0.70

# Tomar una muestra aleatoria del dataset
df_muestra = df.sample(frac=fraccion, random_state=42)


In [13]:
data_type_check(df_muestra)
df_muestra.sample(2)


 Resumen del dataframe:

Dimensiones:  (92351, 2)
       columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0  business_id       100.0      0.0            0    object
1         date       100.0      0.0            0    object


,business_id,date
12361,4x6XkcuyzFHjlV7I7wqF0w,"2012-02-22 01:47:21, 2012-04-27 13:28:41, 2012..."
101577,lNbllJ2epwcSIqaAUEXrxw,"2018-03-19 19:35:38, 2019-04-15 12:02:59, 2019..."



📤 LOAD

In [14]:
# Guardar la muestra como un archivo Parquet
pq.write_table(pa.Table.from_pandas(df_muestra), output_path)

# Verificar el tamaño del archivo resultante
file_size = os.path.getsize(output_path) / (1024 * 1024)  # Convertir a MB
print(f'Tamaño del archivo Parquet reducido: {file_size:.2f} MB')

# Ajustar la fracción si es necesario
if file_size > 100:
    print("El archivo sigue siendo demasiado grande. Reduce la fracción y vuelve a intentarlo.")
elif file_size < 90:
    print("El archivo es menor de 90 MB. Podrías aumentar la fracción.")
else:
    print("El tamaño del archivo es adecuado.")

Tamaño del archivo Parquet reducido: 87.89 MB
El archivo es menor de 90 MB. Podrías aumentar la fracción.


### tip.json

In [15]:
# Ruta del archivo JSON
tip = '../0_Dataset/Data_Sucia/Yelp/tip.json'

# Leer el archivo JSON
df_tip = pd.read_json(tip, lines=True)


In [16]:
data_type_check(df_tip)
df_tip.sample(2)


 Resumen del dataframe:

Dimensiones:  (908915, 5)
            columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0           user_id       100.0      0.0            0          object
1       business_id       100.0      0.0            0          object
2              text       100.0      0.0            0          object
3              date       100.0      0.0            0  datetime64[ns]
4  compliment_count       100.0      0.0            0           int64


,user_id,business_id,text,date,compliment_count
894708,ZE09zaiVZnVz-P245EOHkw,uEWsfftrJ7ukPv1xyMVcrg,Extremely high priced. Not coming back.,2014-07-06 02:30:56,0
740005,1JqUn89C8NvMyLZtRv8prw,1FURjeGJi_LBXcJQg8eskw,Their Hawaiian burger is great it's by the bea...,2021-12-08 05:13:19,0


In [17]:
# Almacenar en formato Parquet, cambiando el sufijo '.json' a '.parquet'
df_tip.to_parquet(tip.replace('.json', '.parquet'))

### user.parquet


In [19]:
# Ruta del archivo Parquet original
user_path = '../0_Dataset/Yelp/user.parquet'

# Leer el archivo Parquet
df = pd.read_parquet(user_path)

# Obtener una muestra aleatoria del 3% del DataFrame (ajustar el tamaño según sea necesario)
sample_df = df.sample(frac=0.03, random_state=42)


In [20]:
data_type_check(sample_df)
sample_df.sample(2)


 Resumen del dataframe:

Dimensiones:  (63168, 22)
               columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0              user_id       100.0      0.0            0    object
1                 name       100.0      0.0            0    object
2         review_count       100.0      0.0            0     int64
3        yelping_since       100.0      0.0            0    object
4               useful       100.0      0.0            0     int64
5                funny       100.0      0.0            0     int64
6                 cool       100.0      0.0            0     int64
7                elite       100.0      0.0            0    object
8              friends       100.0      0.0            0    object
9                 fans       100.0      0.0            0     int64
10       average_stars       100.0      0.0            0   float64
11      compliment_hot       100.0      0.0            0     int64
12     compliment_more       100.0      0.0            0     int64
13  compli

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
233812,tvCILc8B7_noSUKkIsze_Q,Mallory,284,2013-03-27 15:55:54,1138,298,776,"2017,2018,2019,20,20","bwj4gvnEeDrH0LWexKIgpQ, GJKVc86sHbx27PUYkV88wg...",58,...,2,0,0,1,18,22,38,38,22,13
358876,144I0K9Dnn1y96wNiMR_2w,gregory,5,2021-07-29 08:45:40,1,0,1,,None,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:

# Ruta del archivo Parquet de muestra
sample_path = '../0_Dataset/Yelp/user_reducido.parquet'

# Guardar la muestra en un nuevo archivo Parquet
sample_df.to_parquet(sample_path, index=False, compression='snappy')

print(f"Muestra guardada en {sample_path}")

Muestra guardada en ../0_Dataset/Yelp/user_reducido.parquet


___

## Dataset Gogle


 **📂Procesamiento del 1er archivo: `Google Maps/metadata-sitios/review-Florida-`**

### Reviews Florida

#### 📦 **Extraccion** de los datos y primera exploración 
 

🌟 Primero, vamos a convertir los datos de las reseñas de Google Maps del estado de Florida, distribuidos en 11 archivos JSON

In [22]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "../0_Dataset/review-Florida"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
reviews = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        reviews.append(df)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_rev_FL = pd.concat(reviews, ignore_index=True)

In [23]:
data_type_check(df_rev_FL)
df_rev_FL.sample(2)


 Resumen del dataframe:

Dimensiones:  (2850000, 8)
   columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0  user_id      100.00     0.00            0   float64
1     name      100.00     0.00            0    object
2     time      100.00     0.00            0     int64
3   rating      100.00     0.00            0     int64
4     text       62.12    37.88      1079510    object
5     pics        3.66    96.34      2745810    object
6     resp       15.98    84.02      2394601    object
7  gmap_id      100.00     0.00            0    object


,user_id,name,time,rating,text,pics,resp,gmap_id
1071965,1.032315e+20,Yunier Velazquez,1580228259725,5,None,None,None,0x88c2efd165f1e9e5:0x37f1f8c780f4963f
294965,1.076242e+20,Gretchen Booth,1554850678093,5,None,None,None,0x88e5b9ab392ece73:0x1f8770a56f649da0


#### **📤 LOAD**

In [24]:
df_rev_FL_muestra = df_rev_FL.sample(frac=0.25, random_state=1) 
# Se exporta el archivo en formato parquet:
df_rev_FL_muestra.to_parquet("../0_Dataset/Data_Sucia/Google/G_review_FL_reducido.parquet", engine="pyarrow")

### Metadata-sitios

🌟 La "metadata" incluye la información de diversos establecimientos en Google Maps.

📂 Esta información está dividida en 11 archivos JSON, organizados en 3 carpetas para facilitar el procesamiento y almacenamiento.

In [27]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "../0_Dataset/Data_Sucia/Google/metadata-sitios"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
metadata = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df1 = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        metadata.append(df1)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_rev_FL = pd.concat(metadata, ignore_index=True)

In [ ]:
data_type_check(df_rev_FL)
df_rev_FL.sample(2)


 Resumen del dataframe:

Dimensiones:  (2200008, 15)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0               name      100.00     0.00           30    object
1            address       97.09     2.91        64075    object
2            gmap_id      100.00     0.00            0    object
3        description        7.20    92.80      2041504    object
4           latitude      100.00     0.00            0   float64
5          longitude      100.00     0.00            0   float64
6           category       99.37     0.63        13824    object
7         avg_rating      100.00     0.00            0   float64
8     num_of_reviews      100.00     0.00            0     int64
9              price        7.72    92.28      2030122    object
10             hours       72.78    27.22       598803    object
11              MISC       75.59    24.41       536945    object
12             state       74.20    25.80       567655    object
13  relative_results       89.55    

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
414902,Ralph's Barber Shop,"Ralph's Barber Shop, 1827-A, Reynolds Ave, Nor...",0x88fe65e4c7dd302f:0x97afae4dd6103404,None,32.858370,-79.969606,[Barber shop],4.9,28,None,"[[Tuesday, 10AM–6PM], [Wednesday, 10AM–6PM], [...","{'Highlights': ['LGBTQ friendly'], 'Accessibil...",Open ⋅ Closes 6PM,"[0x88fe64e8907df079:0x82bc492c4ace3924, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
1174132,Eyeworks At Midtown,"Eyeworks At Midtown, 316 Gray St, Houston, TX ...",0x8640bf41a157d6cd:0xc10fff1ee4d69980,None,29.751944,-95.377058,"[Eye care center, Optometrist]",4.6,8,None,"[[Thursday, 10AM–5PM], [Friday, Closed], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 10AM,"[0x8640bf6d03eca757:0x19ac5705011d7bf0, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...


📤 LOAD

In [ ]:
df_rev_FL_muestra = df_rev_FL.sample(frac=0.10, random_state=1) 

In [ ]:
# Se exporta el archivo en formato parquet:
df_rev_FL_muestra.to_parquet("../0_Dataset/Data_Sucia/Google/G_metadata_FL_reducido.parquet", engine="pyarrow")
gc.collect()

0

In [ ]:
#  Posibles soluciones: Exportar multiples parquet de metadata y unirlos luego 